In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import pearsonr, wilcoxon

from statsmodels.sandbox.stats.multicomp import multipletests 

# Множественная проверка гипотез
## Примеры задач
Выберите задачи, в которых обязательно нужно применять поправку на множественную проверку гипотез
- Локализация различий в активности мозга испытуемых в разных экспериментальных условиях
- Сравнение эффективности лечения пациентов в подгруппах по большому количеству признаков
- Выбор инвестиционных фондов с помощью сравнения доходности каждого из них с доходностью базового актива по нескольким историческим периодам
- Попарное сравнение средних большого количества выборок

## Классификатор C4.5
Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле: AUC.txt

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [2]:
df = pd.read_csv('AUCs.txt', sep = '\t')
df.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [3]:
pairs = combinations([1,2,3,4],2)
df_pairs = pd.DataFrame(columns = ['left', 'right', 'pvalue'])
for pair in pairs:
    df_left = df.iloc[:, pair[0]]
    df_right = df.iloc[:, pair[1]]
    p_value = wilcoxon(df_left, df_right).pvalue
    df_pairs.loc[len(df_pairs), :] = [df.columns[pair[0]], df.columns[pair[1]], p_value]

In [4]:
df_pairs = df_pairs.sort_values(by = ['pvalue'], ascending = False)

In [5]:
df_pairs.iloc[0]

left          C4.5
right      C4.5+cf
pvalue    0.861262
Name: 1, dtype: object

## Значимые различия
Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [6]:
(df_pairs.pvalue >= 0.05)

1     True
4     True
3    False
5    False
2    False
0    False
Name: pvalue, dtype: bool

## Самый значимый параметр
Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?
- m

## Метод Холма
Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [7]:
reject, p_corrected, a1, a2 = multipletests(df_pairs.pvalue , 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [8]:
print(reject)
print(p_corrected)

[False False False False False False]
[0.861262330095348 0.6556513516892812 0.13899818938018618
 0.09163639741742635 0.07953222050851687 0.06454279871873778]


## Метод Бенджамини-Хохберга
Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [9]:
reject, p_corrected, a1, a2 = multipletests(df_pairs.pvalue , 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [10]:
print(reject)
print(p_corrected)

[False False False  True  True  True]
[0.861262330095348 0.3933908110135687 0.06949909469009309
 0.045818198708713176 0.045818198708713176 0.045818198708713176]


## Метод Бенджамини-Хохберга даёт маху
Насколько корректно, на ваш взгляд, применение метода Бенджамини-Хохберга в этой задаче?
- Некорректно: статистики, соответствующие разным гипотезам, зависимы, так что предположения метода Бенджамини-Хохберга не выполняются

## Что можно улучшить?
Мы подозреваем, что в проведённом с C4.5 эксперименте на самом деле классификаторы сильнее отличаются друг от друга, просто нам не удалось это заметить. Что можно сделать, чтобы увеличить вероятность обнаружения различий, если они действительно существуют?
- Взять больше датасетов

## Результат
Был закреплен материал из лекций. Увидел на практике, как серьёзна бывает порой поправка на множественную проверку гипотез.